In [275]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

In [276]:
# !pip3 install tensorflow

In [277]:
# iris = load_iris()
# X = iris.data
# y = iris.target

In [278]:
# X_train, X_test, y_train, y_test = train_test_split(X,y)

In [279]:
# iris.keys()

In [280]:
drug_train = pd.read_csv("Data/drugLibTrain_raw.tsv",delimiter='\t',encoding='utf-8')
drug_test = pd.read_csv("Data/drugLibTest_raw.tsv",delimiter='\t',encoding='utf-8')

In [281]:
print(drug_train.shape)
print(drug_test.shape)
print(drug_train.shape + drug_test.shape)

(3107, 9)
(1036, 9)
(3107, 9, 1036, 9)


In [282]:
drug=pd.concat([drug_train,drug_test], axis=0, join='outer', ignore_index=False, keys=None,
          levels=None, names=None, verify_integrity=False, copy=True)

In [283]:
drug.head()

,Unnamed: 0,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview
0,2202,enalapril,4,Highly Effective,Mild Side Effects,management of congestive heart failure,slowed the progression of left ventricular dys...,"cough, hypotension , proteinuria, impotence , ...","monitor blood pressure , weight and asses for ..."
1,3117,ortho-tri-cyclen,1,Highly Effective,Severe Side Effects,birth prevention,Although this type of birth control has more c...,"Heavy Cycle, Cramps, Hot Flashes, Fatigue, Lon...","I Hate This Birth Control, I Would Not Suggest..."
2,1146,ponstel,10,Highly Effective,No Side Effects,menstrual cramps,I was used to having cramps so badly that they...,Heavier bleeding and clotting than normal.,I took 2 pills at the onset of my menstrual cr...
3,3947,prilosec,3,Marginally Effective,Mild Side Effects,acid reflux,The acid reflux went away for a few months aft...,"Constipation, dry mouth and some mild dizzines...",I was given Prilosec prescription at a dose of...
4,1951,lyrica,2,Marginally Effective,Severe Side Effects,fibromyalgia,I think that the Lyrica was starting to help w...,I felt extremely drugged and dopey. Could not...,See above


In [425]:
drug['Alltext']=drug['benefitsReview']#+' '+drug['benefitsReview']\
#+' ' +drug['commentsReview']+''+drug['sideEffectsReview']

In [426]:
drug['Alltext'][0:5]

0    slowed the progression of left ventricular dys...
1    Although this type of birth control has more c...
2    I was used to having cramps so badly that they...
3    The acid reflux went away for a few months aft...
4    I think that the Lyrica was starting to help w...
Name: Alltext, dtype: object

In [427]:
drug['Alltext'] = drug['Alltext'].apply(lambda x : str(x))

In [428]:
texts_tr = drug.Alltext

max_words = 500

tokenizer_tr = Tokenizer(num_words=max_words)
# tokenizer_tr = Tokenizer()
tokenizer_tr.fit_on_texts(texts_tr)
# data_tr = tokenizer_tr.texts_to_sequences(texts_tr)
X = tokenizer_tr.texts_to_matrix(texts_tr, mode='binary')

word_index = tokenizer_tr.word_index
print('Found %s unique tokens.' % len(set(word_index))) #88582

Found 9115 unique tokens.


In [429]:
y=drug['rating'].values

In [430]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=2)

In [431]:
# print(drug_train.shape)
# print(drug_test.shape)
# print(drug.shape)

In [432]:
#drug['rating'].nunique()

In [433]:
#drug.head()

In [434]:
#drug.columns

In [435]:
#drug['sideEffects'].unique(),drug['sideEffects'].nunique()

# Gaussian Discriminant Analysis(GDA)

In [436]:
class GaussianDiscriminantAnalysis:
    def __init__(self,epsilon=10e-10):
        self.epsilon=epsilon
        
    def ph(self,y):
        phi=[]
        for k in self.classes:#range(self.nb_class):
            phi.append((np.sum(y==k))/len(y))
        return phi   
    
    def phi(self, y):
        return (np.sum(y==1))/len(y)
    
    def mu_computed(self,X,y):
        list_of_mu=[]
        for k in self.classes: #range(self.nb_class):
            list_of_mu.append((np.sum(X[y==k],axis=0))/np.sum(y==k))
            
        return list_of_mu
    
    def covariance(self,X,y):
        sigma= np.zeros((len(X),len(X)))
        M = np.zeros_like(X)
        for i in range(1,self.nb_class):
            M[y== i]=self.list_mu[i]
        return (((X-M).T)@(X-M))/len(y)
    
    def prob_class(self,X):
        half_len = X.shape[1]/2
        det_cvar = np.sqrt(np.linalg.det(self.cvar ))
        inv_cvar = np.linalg.inv(self.cvar + (self.epsilon*np.eye(len(self.cvar))))
        list_prob=[]
        for i in self.classes:#range(self.nb_class):
            A=((X - self.list_mu[i])@inv_cvar)
            #print('dima',A)
            B=(X-self.list_mu[i])
            #print('dimb',B)
            C=np.sum(np.multiply(A,B),axis=1)
            list_prob.append(np.exp( -0.5*C)*(1/((2*np.pi)**half_len)*det_cvar))
        return list_prob
    
    def train(self, X, y):
        self.nb_class = len(np.unique(y))
        self.classes = np.unique(y)
        self.fi = self.ph(y)
        self.list_mu = self.mu_computed(X,y)
        self.cvar = self.covariance(X, y)
        
    def predict(self, X):        
        proby=self.fi
        list_probclass= self.prob_class(X)
        predict_prob=[]
        for i in self.classes:#range(self.nb_class):
            
            predict_prob.append(list_probclass[i]*proby[i])
        
   
        return np.argmax(np.array(predict_prob).T,axis=1) 

In [437]:
#GDA = GaussianDiscriminantAnalysis()

In [438]:
#GDA.train(X_train,y_train)

In [439]:
#X_test.shape

In [440]:
#prediction=GDA.predict(X_test)

In [441]:
#np.sum(prediction == y_test) / len(y_test)*100

In [442]:
# np .random.seed(0)

# data = np.random.rand(200, 4)
# data = np.round(data, decimals=0)
# target = np.random.randint(0, 3, size=(200, 1))
# data=np.concatenate((data,target),axis=1)
# X_train = data[:150, :-1]
# y_train = data[:150, -1]
# X_test = data[150:, :-1]
# y_test = data[150:, -1]

In [443]:
#y_train

In [444]:
data[:100,-1].shape

(100,)

In [445]:
2*np.ones(100).shape

(100, 100)

In [446]:
#np.random.((200,1))

In [447]:
y_train

array([ 3,  1,  8, ...,  5,  8, 10])

#  Naive Bayes (NB)

In [448]:
# text = drug[['commentsReview','benefitsReview','sideEffectsReview']].apply(lambda x: 
#                                                     ' '.join(str(s) for s in x), axis=1)
# c=CountVectorizer(max_features=5,binary=True)
# j = c.fit_transform(text)
# X = j.toarray()
# y=drug['rating']
# X_train, X_test, y_train, y_test = train_test_split(X,y)

In [449]:
import ipdb #ipython debogeur

In [450]:
class BernoulliNaiveBayes:
    
    def __init__(self):
        pass
        
        
    def phiy(self,y): 
        phi=[]
        for k in self.classes:
            phi.append(np.sum(y==k)/len(y))
        return phi
    
    
    def phix1y(self,X,y):
        phi_list=[]
        for k in self.classes:
            phix=[]
            for i in range(X.shape[1]):
                X_i=X[:,i]
                select_y=X_i[y==k]
                phix.append((np.sum(select_y==1)+1)/(np.sum(y==k)+self.nb_class))
            phi_list.append(phix)
        return phi_list
    
    def phix0y(self,X,y):
        phi_list=[]
        for k in self.classes:
            phix=[]
            for i in range(X.shape[1]):
                X_i=X[:,i]
                select_y=X_i[y==k]
                phix.append((np.sum(select_y==0)+1)/(np.sum(y==k)+self.nb_class))
            phi_list.append(phix)
        return phi_list
    
    def probabxy(self,X):
        
        probxyk=[]
        arr1=np.zeros_like(X)
#         ipdb.set_trace()
        for k in range(self.nb_class):
            probxy1=[]
            for i in range(X.shape[0]):
                X_i=X[i,:]
                prob=1
                for j in range(len(X_i)):

                    if X_i[j]==1:
                        prob *=  self.fix1y[int(k)][j]
                    else:
                        
                        prob *= self.fix0y[int(k)][j]
                    arr1[i,j]=prob
                probxy1.append(prob)
            #print(np.array(probxy1).shape)
            probxyk.append(probxy1)
        return  probxyk 

    def fit_train(self, X,y):
        self.nb_class = len(np.unique(y))
        self.classes = np.unique(y)
        self.fiy = self.phiy(y)
        self.fix1y = np.array(self.phix1y(X,y))
        self.fix0y = np.array(self.phix0y(X,y))
        #print(self.fix1y)
        #print(self.fix0y[0:10])
        
    def predict(self,X) :
        proby=self.fiy
        list_probclass= self.probabxy(X)
        
        predict_prob=[]
        predict_prob=np.array(list_probclass).T*proby
        if 0 in self.classes:
            return  np.argmax(predict_prob,axis=1)
        else:
            return  np.argmax(predict_prob,axis=1)+1         
        
        
        #return  np.argmax(predict_prob,axis=1)

In [451]:

NB = BernoulliNaiveBayes()
NB.fit_train(X_train,y_train)

In [452]:
prediction=NB.predict(X_test)

In [453]:

np.sum(prediction==y_test)/len(y_test)

0.2629674306393245

In [454]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

In [455]:
clf = MultinomialNB()

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [456]:
prediction=clf.predict(X_test)

In [457]:

np.sum(prediction==y_test)/len(y_test)

0.24728588661037396

array([0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1.,
       1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 1., 0., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1.,
       0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 0.,
       1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1.,
       0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0.,
       1., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       1., 1., 0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1.])

In [69]:
#NB.fit_train(X_train,y_train)

In [70]:
#prediction=NB.predict(X_test)

In [71]:
#prediction

In [72]:
#y_test

In [73]:
#np.sum(prediction==y_test)/len(y_test)*100